In [3]:
import optuna
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, make_scorer

In [4]:
train=pd.read_csv('/kaggle/input/fsajhd/df_train_clean.csv')
test=pd.read_csv('/kaggle/input/fsajhd/df_test_clean.csv')
sub=pd.read_csv('/kaggle/input/fsajhd/sample_submission.csv')

In [5]:
tarain=pd.read_csv('/kaggle/input/fsajhd/train.csv')

In [6]:
tarain.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200
1,1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999
2,2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900
3,3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000
4,4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500


In [7]:
train.head(5)

,brand,milage,fuel_type,transmission,ext_col,int_col,accident,clean_title,price,Horsepower,Displacement,Cylinder Count,model_age
0,31,213000,2,0,312,71,2,1,4200,172.0,1.6,4.0,18
1,28,143250,2,0,263,10,0,1,4999,252.0,3.9,8.0,23
2,9,136731,1,0,38,71,2,1,13900,320.0,5.3,8.0,23
3,16,19500,2,2,29,14,2,1,45000,420.0,5.0,8.0,8
4,36,7388,2,0,29,10,2,1,97500,208.0,2.0,4.0,4


In [8]:
test.sample(5)

,brand,milage,fuel_type,transmission,ext_col,int_col,accident,clean_title,Horsepower,Displacement,Cylinder Count,model_age
51759,14,59715,1,0,128,71,2,1,290.000000,3.5,6.000000,8
105377,43,40570,2,2,38,14,0,1,440.000000,4.8,8.000000,10
109945,43,12758,2,0,304,14,2,1,400.000000,3.8,6.000000,11
97973,27,53782,2,0,234,31,2,1,295.000000,3.5,6.000000,5
30121,22,3752,2,0,31,14,2,1,244.553252,2.0,6.374268,3


In [9]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188533 entries, 0 to 188532
Data columns (total 13 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   brand           188533 non-null  int64  
 1   milage          188533 non-null  int64  
 2   fuel_type       188533 non-null  int64  
 3   transmission    188533 non-null  int64  
 4   ext_col         188533 non-null  int64  
 5   int_col         188533 non-null  int64  
 6   accident        188533 non-null  int64  
 7   clean_title     188533 non-null  int64  
 8   price           188533 non-null  int64  
 9   Horsepower      188533 non-null  float64
 10  Displacement    188533 non-null  float64
 11  Cylinder Count  188533 non-null  float64
 12  model_age       188533 non-null  int64  
dtypes: float64(3), int64(10)
memory usage: 18.7 MB


## Cat Boost

In [10]:
# import optuna
# import numpy as np
# import pandas as pd
# from sklearn.model_selection import train_test_split, cross_val_score, KFold
# from sklearn.preprocessing import StandardScaler
# from sklearn.pipeline import Pipeline
# from sklearn.compose import ColumnTransformer
# from catboost import CatBoostRegressor
# from sklearn.metrics import mean_squared_error, make_scorer

# # Load dataset
# X = train.drop(columns=['price'])
# y = train['price']

# # Split dataset
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# # Define numerical and categorical columns
# num_cols = ['milage', 'Horsepower', 'Displacement', 'Cylinder Count', 'model_age']
# cat_cols = ['brand', 'fuel_type', 'transmission', 'ext_col', 'int_col', 'accident', 'clean_title']

# # Preprocessing (Standard Scaling for numerical features)
# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num', StandardScaler(), num_cols)
#     ],
#     remainder='passthrough'
# )

# # RMSE scorer for optimization
# rmse_scorer = make_scorer(mean_squared_error, squared=False)

# # Define objective function for Optuna
# def objective(trial):
#     params = {
#         'iterations': trial.suggest_int('iterations', 100, 1000, step=100),
#         'depth': trial.suggest_int('depth', 3, 10),
#         'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
#         'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-3, 1e-1, log=True),
#         'random_seed': 42,
#         'loss_function': 'RMSE',
#         'verbose': 0
#     }

#     # Create pipeline
#     pipeline = Pipeline(steps=[
#         ('preprocessor', preprocessor),
#         ('model', CatBoostRegressor(**params))
#     ])

#     # Perform cross-validation
#     kf = KFold(n_splits=5, shuffle=True, random_state=42)
#     scores = cross_val_score(pipeline, X_train, y_train, cv=kf, scoring=rmse_scorer)
    
#     return np.mean(scores)

# # Run Optuna optimization
# study = optuna.create_study(direction='minimize')
# study.optimize(objective, n_trials=50)

# # Get best parameters
# best_params = study.best_params
# print("Best Hyperparameters:", best_params)

# -------------------------------------Output---------------------------------------------------

# Best Hyperparameters: {'iterations': 500,
#                        'depth': 9,
#                        'learning_rate': 0.012904818994335723,
#                        'l2_leaf_reg': 0.009754632381874578}

In [11]:
train.head(
)

,brand,milage,fuel_type,transmission,ext_col,int_col,accident,clean_title,price,Horsepower,Displacement,Cylinder Count,model_age
0,31,213000,2,0,312,71,2,1,4200,172.0,1.6,4.0,18
1,28,143250,2,0,263,10,0,1,4999,252.0,3.9,8.0,23
2,9,136731,1,0,38,71,2,1,13900,320.0,5.3,8.0,23
3,16,19500,2,2,29,14,2,1,45000,420.0,5.0,8.0,8
4,36,7388,2,0,29,10,2,1,97500,208.0,2.0,4.0,4


In [12]:
import joblib
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error

# Load dataset
X = train.drop(columns=['price'])
y = train['price']

# Define numerical and categorical columns
num_cols = ['milage', 'Horsepower', 'Displacement', 'Cylinder Count', 'model_age']
cat_cols = ['brand', 'fuel_type', 'transmission', 'ext_col', 'int_col', 'accident', 'clean_title']

# Updated hyperparameters from Optuna with GPU support
tuned_params = {
    'iterations': 500,
    'depth': 9,
    'learning_rate': 0.012904818994335723,
    'l2_leaf_reg': 0.009754632381874578,
    'random_seed': 42,
    'loss_function': 'RMSE',
    'verbose': 100,
    'task_type': 'GPU'
}

# Define preprocessing pipeline
num_pipeline = Pipeline(steps=[('scaler', StandardScaler())])
preprocessor = ColumnTransformer(
    transformers=[
        ('num_pipeline', num_pipeline, num_cols)
    ],
    remainder='passthrough'
)

# Number of folds for cross-validation
n_folds = 5
kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)

oof_preds = np.zeros(len(X))

oof_df = pd.DataFrame(columns=['ID', 'Actual', 'OOF_Pred_CatBoost', 'Fold'])

for fold, (train_idx, val_idx) in enumerate(kf.split(X, y), start=1):
    print(f"\nTraining Fold {fold}/{n_folds}...")
    
    X_tr, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_tr, y_val = y.iloc[train_idx], y.iloc[val_idx]
    
    # Create pipeline with preprocessor and CatBoost model
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', CatBoostRegressor(**tuned_params))
    ])
    
    # Train model
    pipeline.fit(X_tr, y_tr)
    
    # Predict on validation set
    y_val_pred = pipeline.predict(X_val)
    oof_preds[val_idx] = y_val_pred
    
    # Compute fold RMSE
    fold_rmse = mean_squared_error(y_val, y_val_pred, squared=False)
    print(f"Fold {fold} RMSE: {fold_rmse:.4f}")
    
    # Store fold results in DataFrame
    fold_df = pd.DataFrame({
        'ID': X.index[val_idx],
        'Actual': y_val.values,
        'OOF_Pred_CatBoost': y_val_pred,
        'Fold': fold
    })
    
    oof_df = pd.concat([oof_df, fold_df], ignore_index=True)

# Compute overall RMSE
oof_rmse = mean_squared_error(y, oof_preds, squared=False)
print(f"\nOverall OOF RMSE: {oof_rmse:.4f}")

# Save OOF predictions
oof_df.to_csv('oof_predictions_catboost.csv', index=False)
print("OOF predictions saved to 'oof_predictions_catboost.csv'.")

# Save trained model
joblib.dump(pipeline, 'catboost_model.pkl')
print("Model saved as 'catboost_model.pkl'.")

# -------------------------------------Output---------------------------------------------------

# Overall OOF RMSE: 73119.5593


Training Fold 1/5...
0:	learn: 79713.4689556	total: 13.2s	remaining: 1h 49m 38s
100:	learn: 73936.8051043	total: 13.7s	remaining: 54.2s
200:	learn: 72348.2506064	total: 14.2s	remaining: 21.2s
300:	learn: 71219.9449981	total: 14.8s	remaining: 9.77s
400:	learn: 70197.0594922	total: 15.3s	remaining: 3.78s
499:	learn: 69323.1953476	total: 15.8s	remaining: 0us
Fold 1 RMSE: 68496.3872

Training Fold 2/5...


<ipython-input-12-9671251e589d>:79: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  oof_df = pd.concat([oof_df, fold_df], ignore_index=True)


0:	learn: 79648.8700702	total: 7.01ms	remaining: 3.5s
100:	learn: 73849.9914587	total: 519ms	remaining: 2.05s
200:	learn: 72175.2809056	total: 1.03s	remaining: 1.53s
300:	learn: 71105.5906519	total: 1.55s	remaining: 1.03s
400:	learn: 70121.0897965	total: 2.09s	remaining: 517ms
499:	learn: 69195.9723134	total: 2.61s	remaining: 0us
Fold 2 RMSE: 68968.5890

Training Fold 3/5...
0:	learn: 78476.9778863	total: 5.86ms	remaining: 2.92s
100:	learn: 72637.7886151	total: 516ms	remaining: 2.04s
200:	learn: 70926.7991053	total: 1.03s	remaining: 1.53s
300:	learn: 69750.0222755	total: 1.56s	remaining: 1.03s
400:	learn: 68776.1554154	total: 2.09s	remaining: 517ms
499:	learn: 67904.5640466	total: 2.62s	remaining: 0us
Fold 3 RMSE: 74131.5944

Training Fold 4/5...
0:	learn: 77748.9284480	total: 5.85ms	remaining: 2.92s
100:	learn: 71808.2206838	total: 516ms	remaining: 2.04s
200:	learn: 70079.5371286	total: 1.04s	remaining: 1.54s
300:	learn: 68985.1757732	total: 1.55s	remaining: 1.02s
400:	learn: 68013.81

In [13]:
final_test_predictions = pipeline.predict(test)

# Save submission file
sub = pd.read_csv('/kaggle/input/fsajhd/sample_submission.csv')
sub['price'] = final_test_predictions
sub.to_csv('CatBoost_submission.csv', index=False)
print("Submission file saved as 'lrsubmission.csv'.")
sub.head()


Submission file saved as 'lrsubmission.csv'.


,id,price
0,188533,15935.550664
1,188534,78654.810785
2,188535,50620.511109
3,188536,23767.392186
4,188537,30608.588917
